In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

In [ ]:
%load_ext Cython
%load_ext line_profiler
%load_ext memory_profiler

In [ ]:
from tmqrfeed.manager import DataManager
from tmqrindex.index_exo_base import IndexEXOBase
from datetime import datetime

In [ ]:
from bdateutil import relativedelta
from tmqr.logs import log
from tmqrfeed.quotes import QuoteContFut
from tmqrfeed import Costs


class EXOSpreadIndex(IndexEXOBase):
    _description_short = "EXO Vanilla Long/Short spread index"
    _description_long = ""

    _index_name = "EXOSpreadFixed"
    
    def __init__(self, datamanager, **kwargs):
        super().__init__(datamanager, **kwargs)
        
        self.PRIMARY_INSTRUMENT = 'US.ES'
        self.SECONDARY_INSTRUMENT = 'US.CL'
              
        self.costs_futures = 3.0
        self.costs_options = 3.0
    
    def setup(self):
        
        #
        # IMPORTANT! Use trading session of self.PRIMARY_INSTRUMENT 
        #   All US.CL quotes and positions will use 'US.ES' decision and execution time
        # 
        self.dm.session_set(self.PRIMARY_INSTRUMENT, session_instance=self.session)
        
        #
        # Set primary quotes for 'US.ES' to align all data to its index
        #
        self.dm.series_primary_set(QuoteContFut, self.PRIMARY_INSTRUMENT,
                                   timeframe='D', decision_time_shift=self.decision_time_shift)
        
        self.dm.series_extra_set(self.SECONDARY_INSTRUMENT, QuoteContFut, self.SECONDARY_INSTRUMENT,
                                   timeframe='D', decision_time_shift=self.decision_time_shift)
        #
        # Set index costs (costs are calculated at the final stage, of index equirt calculation)
        # 
        self.dm.costs_set(self.PRIMARY_INSTRUMENT.split('.')[0], Costs(per_contract=self.costs_futures,
                                                               per_option=self.costs_options))
        
    
    def calc_exo_logic(self):
        """
        Calculates SmartEXO logic.
        NOTE: this method must use self.dm.quotes() or self.dm.quotes(series_key='for_secondary_series') to 
              calculate SmartEXO logic
        :return: Pandas.DataFrame with index like in dm.quotes() (i.e. primary quotes)
        """
        pass
    
    def manage_position(self, dt, pos, logic_df):
        """
        Manages opened position (rollover checks/closing, delta hedging, etc)
        :param dt: current datetime
        :param pos: Position instance
        :param logic_df: result of calc_exo_logic()[dt]  if applicable
        :return: nothing, manages 'pos' in place
        """        
        #
        # Check expiration moment
        #
        if pos.almost_expired_ratio(dt) > 0:
            pos.close(dt)
                

    def construct_position(self, dt, pos, logic_df):
        """
        EXO position construction method
        
        NOTE!: this method only called when there is no active position for 'dt'
        :param dt: current datetime
        :param pos: Position instance
        :param logic_df: result of calc_exo_logic()[dt]  if applicable
        :return: nothing, manages 'pos' in place
        """
        
        fut_primary, opt_chain_primary = self.dm.chains_options_get(self.PRIMARY_INSTRUMENT, dt)
        fut_secondary, opt_chain_secondary = self.dm.chains_options_get(self.SECONDARY_INSTRUMENT, dt)
        
        
        # PRIMARY long
        #pos.add_transaction(dt, fut_primary, 1.0)
        # we can use options too!
        #pos.add_transaction(dt, opt_chain_primary.find(dt, 0.15, 'C', how='delta'), 1.0)
        #pos.add_transaction(dt, opt_chain_primary.find(dt, 0.55, 'P', how='delta'), -1.0)
        
        pos.add_transaction(dt, opt_chain_primary.find(dt, 0.45, 'P', how='delta'), -1.0)
        pos.add_transaction(dt, opt_chain_primary.find(dt, 0.35, 'C', how='delta'), 1.0)
       
        
        # SECONDARY short
        #pos.add_transaction(dt, fut_secondary, -1.0)
        # we can use options too!
        #pos.add_transaction(dt, opt_chain_secondary.find(dt, 0.25, 'P', how='delta'), -1.0)

In [ ]:
dm = DataManager(date_start=datetime(2011, 5, 1))

In [ ]:
INDEX_CONTEXT = {
    'instrument': "US.ES",
    'costs_futures': 3.0,
    'costs_options': 3.0,
}
index = EXOSpreadIndex(dm, **INDEX_CONTEXT)

#
# BOTH index init code lines are equal
#

#index = EXODeltaTargetGeneric(dm, instrument="US.ES", costs_futures=3.0, costs_options=3.0)

In [ ]:
index.run()


In [ ]:
index.index_name

## Index equity

In [ ]:
index.data['equity_execution'].plot()
title(index.index_name)

# Spread position

Both of ES and CL have the same decision time and price



In [ ]:
index.position

# Save index

In [ ]:
index.save()

In [ ]:
index.position